In [ ]:
from bs4 import BeautifulSoup
import os 
import urllib
import json
import pandas as pd
import tensorflow as tf 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
def read(files):
  for filename in files:
    filename = '/content/gdrive/MyDrive/docs/' + filename      
    with open(filename) as open_file:
          content = open_file.read()
          soup = BeautifulSoup(content, "xml")
          print(soup.prettify())
          print("First AGENCY tag in " + filename[len(path) + 1:] + ": " + str(soup.find('AGENCY')))

In [ ]:
def extract_agency(infile):
  contents = infile.read()
  soup = BeautifulSoup(contents, 'xml')
  agencies = soup.find_all('AGENCY')
  for agency in agencies:
    print(agency.get_text())


def extract_time(infile):
  contents = infile.read()
  soup = BeautifulSoup(contents, 'xml')
  time = soup.find('EFFDATE')
  if time is not None:
    time = time.text
    start_idx = time.find("<P>")
    time = time[start_idx + 1:]
    end_idx = time.find("<P>")
    time = time[0:end_idx]
    time = time.split("\n")[2]
  print(time)


def extract_subject(infile):
  contents = infile.read()
  soup = BeautifulSoup(contents, 'xml')
  subject = soup.find('SUBJECT')
  print(subject)


In [ ]:
def get_json(url, tags):
  values = []
  with urllib.request.urlopen(url) as response:
    json_file = response.read()
    j = json.loads(json_file)
    for i in tags:
      values.append(j[i])
    
  return values

# title and publication_date

def populate_df(files, tags):
  all_values = []
  for i in files:
    fname = i.replace(".xml", ".json")
    v = get_json("https://www.federalregister.gov/api/v1/documents/" + fname, tags)
    all_values.append(v)
  
  return all_values


files = os.listdir('/content/gdrive/MyDrive/docs/')
t = ["title", "abstract", "publication_date", "topics"]
s = populate_df(files, t)
d = {}
d["filename"] = files

for i in range(0, len(t)):
  col = [row[i] for row in s]
  d[t[i]] = col

df = pd.DataFrame(d)

print(df)

df.to_csv("fed_doc_data_abs.csv", index=False)

            filename  ...                                             topics
0     2014-21224.xml  ...  [Accounting, Administrative practice and proce...
1     2020-08603.xml  ...  [Business and industry, Cooperatives, Reportin...
2     2020-04776.xml  ...  [Reporting and recordkeeping requirements, Sec...
3        02-5778.xml  ...  [Commodity futures, Reporting and recordkeepin...
4     2020-25747.xml  ...            [Administrative practice and procedure]
...              ...  ...                                                ...
3613  2011-28313.xml  ...  [Administrative practice and procedure, Confli...
3614   2012-6799.xml  ...                                                 []
3615  2011-18777.xml  ...  [Advertising, Brokers, Commodity futures, Cons...
3616  2011-11551.xml  ...  [Advertising, Brokers, Commodity futures, Cons...
3617      01-175.xml  ...  [Administrative practice and procedure, Author...

[3618 rows x 5 columns]
